# Chart Update Script

**MUST be stored in ROOT of chartstore repo (folder)**

In [12]:
# OPTIONAL - uncomment to use
# Update hongxiongmao from Github (only need if recent changes worth updating)
! pip install git+https://github.com/djmcnay/hongxiongmao#egg=hongxiongmao --upgrade

  Cloning https://github.com/djmcnay/hongxiongmao to /private/var/folders/6g/z_d85_5j1nxb1w_b3h9hzgz40000gn/T/pip-install-qouit_wf/hongxiongmao
  Running command git clone -q https://github.com/djmcnay/hongxiongmao /private/var/folders/6g/z_d85_5j1nxb1w_b3h9hzgz40000gn/T/pip-install-qouit_wf/hongxiongmao
  Stored in directory: /private/var/folders/6g/z_d85_5j1nxb1w_b3h9hzgz40000gn/T/pip-ephem-wheel-cache-vf3cpux6/wheels/40/f0/13/cb7f72f4fee1e8b65e0e31689c582a05d9d84d7f2b7b07cc16
Successfully built hongxiongmao
  Found existing installation: hongxiongmao 0.1.dev0
    Uninstalling hongxiongmao-0.1.dev0:
      Successfully uninstalled hongxiongmao-0.1.dev0


In [13]:
import pickle
import numpy as np
import pandas as pd
import datetime

# Plotly Stuff
import plotly.offline as py
py.init_notebook_mode(connected=True)

# HXM Dependencies
import hongxiongmao as hxm
from hongxiongmao import utilities

In [14]:
# POKEMON
p = hxm.pokemon.pokemon_go()
p.download_and_run()
p.plotlyplot()
py.plot(p.plot, filename='PlotlyHTMLexJS/pokemon.html',
        auto_open=False, include_plotlyjs='cdn', include_mathjax='cdn')

'PlotlyHTMLexJS/pokemon.html'

In [15]:
# Pull US ETF Data from AlphaVantage
dl = hxm.download.alphavantage()    # setup download class for av
us_etfs = dl.dl_from_ticker_dict(dl.us_etfs, output_size='full')   # pull 100d of daily data

AlphaVantage call on ACWI downloaded
AlphaVantage call on SPY downloaded
AlphaVantage call on QQQ downloaded
AlphaVantage call on VEA downloaded
AlphaVantage call on IEMG downloaded
ERR: AlphaVantage call on VWO; retry in 5s
ERR: AlphaVantage call on VWO; retry in 10s
AlphaVantage call on VWO downloaded
AlphaVantage call on ASHR downloaded
AlphaVantage call on VNQ downloaded
AlphaVantage call on AGG downloaded
AlphaVantage call on GOVT downloaded
ERR: AlphaVantage call on TIP; retry in 5s
ERR: AlphaVantage call on TIP; retry in 10s
ERR: AlphaVantage call on TIP; retry in 15s
ERR: AlphaVantage call on TIP; retry in 20s
AlphaVantage call on TIP downloaded
AlphaVantage call on LQD downloaded
AlphaVantage call on HYG downloaded
AlphaVantage call on EMB downloaded
AlphaVantage call on EMLC downloaded
ERR: AlphaVantage call on GLD; retry in 5s
ERR: AlphaVantage call on GLD; retry in 10s
ERR: AlphaVantage call on GLD; retry in 15s
ERR: AlphaVantage call on GLD; retry in 20s
AlphaVantage call 

In [16]:
# Principal Drivers Index
etfs=['SPY','VEA','VWO','GOVT','TIP','LQD','HYG','EMB','EMLC','GLD']
px = utilities.daily2weekly(us_etfs['close']).loc[:,etfs]

pdi = hxm.correlation.pdi(ts=px)
pdi.run(n=26)
py.plot(pdi.plot, filename='PlotlyHTMLexJS/principaldrivers.html',
        auto_open=False, auto_play=False, include_plotlyjs='cdn', include_mathjax='cdn')

'PlotlyHTMLexJS/principaldrivers.html'

In [17]:
# Duo-Dendrogram (uses wider ETF universe than PDI)
rtns = utilities.daily2weekly(us_etfs['close']).pct_change()
fig = hxm.correlation.duo_dendrogram_mpl(rtns, px=False)
fig.savefig('matplotlib/duodenrogram_etfs.png', bbox_inches='tight')

In [21]:
# OECD Composite Leading Indicators

## Download OECD CLI & Normalised GDP Data
dl = hxm.download.quandl_hxm()
cli = dl.from_tickerdict('oecd_cli', start_date='-30y') - 100
gdp = dl.from_tickerdict('oecd_normalised_gdp', start_date='-30y') - 100

## Setup Definitions & Groups to be shown on charts
cli_dict = dict(dm=['US', 'UK', 'EURO', 'JAPAN', 'CANADA', 'KOREA', 'OECD'],
                em=['CHINA', 'MEXICO', 'RUSSIA', 'INDIA', 'BRAZIL', 'TURKEY', 'ZAR', 'ASIA5'],)

cli_groups = dict(dm=dict(ALL=cli_dict['dm'], Major=['US', 'EURO', 'UK', 'JAPAN'], Other=['CANADA', 'KOREA', 'OECD']),
                  em=dict(ALL=cli_dict['em'], BRIC=['CHINA', 'BRAZIL', 'RUSSIA', 'INDIA'],
                          Asia=['CHINA', 'INDIA', 'ASIA4'], Latam=['MEXICO', 'BRAZIL'],
                          CEEMEA=['RUSSIA', 'TURKEY', 'ZAR']),)

# Iterate through each grouping in CLI Dict
for x in cli_dict.keys():
    
    # Standard Timeseries Scatterplot using CLI Data
    py.plot(overplot.scatter_from_dataframe(cli.loc[:,cli_dict[x]], groups=cli_groups[x],
                                      title='OECD Normalised CLI - Developed Markets',
                                      ytitle='Normalised CLI (minus 100)',
                                      button_position=[1.2, 0.5, 0, 0, 0, 0], button_direction = 'down'),
            filename='PlotlyHTMLexJS/cli_'+x+'.html',
            auto_open=False, include_plotlyjs='cdn', include_mathjax='cdn')
    
    # Swirlygram - Static
    py.plot(overplot.swirlygram(cli.loc[:,cli_dict[x]], minax=[1, 1],
            title='OECD Normalised CLI - Developed Markets', ytitle='Normalised CLI (minus 100)',),
            filename='PlotlyHTMLexJS/cli_'+x+'_swirlygram.html',
            auto_open=False, auto_play=False, include_plotlyjs='cdn')

# Animated Swirlygram
swirlyregions = ['OECD', 'US']
py.plot(overplot.swirlygram(df = cli.loc[:,swirlyregions], 
                            df2 = gdp.loc[:,swirlyregions],
                            n=6, minax=[1, 1], animation=True,
                            title='OECD CLI vs. GDP; Normalised with 6m lead', ytitle='Normalised CLI (minus 100)',),
        filename='PlotlyHTMLexJS/cli_animated_swirlygram.html',
        auto_open=False, auto_play=False, include_plotlyjs='cdn')

'PlotlyHTMLexJS/cli_animated_swirlygram.html'

## Push to Github Repo

In [22]:
! git add ./*                  # add all files to git staging area
! git commit -m "auto update"
! git push origin master       # push to master branch

The following paths are ignored by one of your .gitignore files:
__pycache__
Use -f if you really want to add them.
[master 1c72eb0] auto update
 6 files changed, 63 insertions(+), 55 deletions(-)
 rewrite PlotlyHTMLexJS/cli_animated_swirlygram.html (65%)
Counting objects: 9, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 9.05 KiB | 3.01 MiB/s, done.
Total 9 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/djmcnay/pandachartstore.git
   ca21f0d..1c72eb0  master -> master
